In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_EOP_new as abatement
os.chdir(directory['curr'])

The file_gams_py_gdb2.gdx is still active and was not deleted.


*Specify settings:*

In [2]:
name = 'A3'
# catalogue = 'dors_2'
catalogue = 'C2'
f = {'tech': directory['data']+f"/techdata_{catalogue}.xlsx",
     'other': directory['data']+f"/Othdata_{catalogue}.xlsx"}
s = ['inputdisp','endofpipe','inputprices']
m = ['ID','EOP'] 
e = {'M': 'vars','M_sets': 'sets'} # dictionary used to load emissions data
other_trees = {'ID_Y': {'s': 'Y', 'read_data': {'Y': 'vars'}, 'Q2P': 'Q2P'}} # the key = tree name, s = sheet that specifies tree structure, read_data (if not None) specifies how to load data.

In [3]:
type_f = {'ID_EC': 'CES_norm',
          'ID_CU': 'MNL',
          'ID_TU': 'CET_norm',
          'ID_TX': 'CES',
          'ID_BU': 'linear_out',
          'ID_BX': 'CES',
          'EOP_CU': 'MNL',
          'EOP_TU': 'CET_norm',
          'EOP_TX': 'CES'}

*Set up nesting tree:*

In [4]:
%run ATree.ipynb

In [5]:
dbs = {}

## 1: Model without EOP

*Initialize without EOP:*

In [6]:
gm = abatement.abate(nts=nts,tech=tech,work_folder=directory['work'],use_EOP=False,**{'data_folder':directory['gams'],'name': name})
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False) 

*Provide initial values:*

In [7]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

*Add some extra options:*

In [8]:
with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as f:
    kwargs_mi = {'opt_file': f.read()}

### 1.1: Solve with leontief values:

In [9]:
gm.write_and_run(name='ID',kwargs_mi=kwargs_mi)
dbs['ID'] = gm.model_instances['ID'].out_db

### 1.2: Adjust elasticities away from leontief case:

In [16]:
db_star = DataBase.GPM_database()
db_star['eta'] = pd.Series(-0.5, index = gm.get('eta').index,name='eta')
db_star['sigma'] = pd.Series(1, index = gm.get('kno_ID_CU').union(gm.get('kno_ID_BX')), name = 'sigma')
db_star['mu'] = pd.Series(1, index = gm.get('map_ID_CU'),name='mu')

*Adjust parameters in a sneaky solve:*

In [17]:
gm.model_instances['ID'].solve_sneakily(db_star=db_star,kwargs_shock={'n_steps':100})
dbs['ID'] = gm.model_instances['ID'].out_db

### 1.3: Calibrate model

In [18]:
gm.model.database['gamma_tau'] = pd.Series(1,index = gm.get('gamma_tau').index,name='gamma_tau')

*Read back solution to main db:*

In [19]:
gm.model.database.merge_dbs(gm.model.database,dbs['ID'],'second')

In [23]:
gm.reset_settings()
gm.setstate('ID_calibrate',init=False)
gm.write_and_run(name='ID_calibrate',kwargs_init={'check_variables':True},options_run={'output':sys.stdout},overwrite=True,kwargs_mi=kwargs_mi)
db_ID_C = gm.model_instances['ID_calibrate'].out_db

--- Job CollectAndRun.gmy Start 09/14/21 11:09:14 29.1.0 rbb4180b WEX-WEI x86 64bit/MS Windows
--- GAMS Parameters defined
    LP CONOPT4
    MIP CPLEX
    RMIP CONOPT4
    NLP CONOPT4
    MCP PATH
    MPEC NLPEC
    RMPEC NLPEC
    CNS CONOPT4
    DNLP CONOPT4
    RMINLP CONOPT4
    MINLP DICOPT
    QCP CONOPT4
    MIQCP SBB
    RMIQCP CONOPT4
    EMP JAMS
    Input C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\CollectAndRun.gmy
    Output C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo3.lst
    ScrDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\225c\
    SysDir C:\GAMS\win64\29.1\
    CurDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\
    LogOption 3
    LogFile C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo3.log
    GDX C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gdb3.gdx
    --ID_0 ID_0
    --shock shock
GAMS 29.1.0   Copyright (C) 1987-2019 GAMS Development. All rights reserved
Licensee: Medium MUD - 

## 2: Solve model

In [8]:
# gm.model.database['testminobj'] = 0;

#### 2.1: EOP

*Set up baseline model and solve:*

In [10]:
gm.write_and_run(name='EOP',kwargs_mi=kwargs_mi)
db_EOP = gm.model_instances['EOP'].out_db

*Read back solution to main db:*

In [11]:
gm.model.database.merge_dbs(gm.model.database,db_EOP,'second')

#### 2.2: ID calibrate

In [12]:
gm.reset_settings()
gm.setstate('ID_calibrate',init=False)
gm.write_and_run(name='ID_calibrate',kwargs_init={'check_variables':True},options_run={'output':sys.stdout},overwrite=True,kwargs_mi=kwargs_mi)
db_ID_C = gm.model_instances['ID_calibrate'].out_db

--- Job CollectAndRun.gmy Start 09/13/21 17:39:06 29.1.0 rbb4180b WEX-WEI x86 64bit/MS Windows
--- GAMS Parameters defined
    LP CONOPT4
    MIP CPLEX
    RMIP CONOPT4
    NLP CONOPT4
    MCP PATH
    MPEC NLPEC
    RMPEC NLPEC
    CNS CONOPT4
    DNLP CONOPT4
    RMINLP CONOPT4
    MINLP DICOPT
    QCP CONOPT4
    MIQCP SBB
    RMIQCP CONOPT4
    EMP JAMS
    Input C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\CollectAndRun.gmy
    Output C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo1.lst
    ScrDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\225a\
    SysDir C:\GAMS\win64\29.1\
    CurDir C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\
    LogOption 3
    LogFile C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo1.log
    GDX C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gdb1.gdx
    --ID_0 ID_0
GAMS 29.1.0   Copyright (C) 1987-2019 GAMS Development. All rights reserved
Licensee: Medium MUD - 10 User License   

#### 2.3: EOP calibrate

*Change to EOP calibration state and resolve:*

In [12]:
gm.reset_settings()
gm.setstate('EOP_calibrate',init=False)
gm.write_and_run(name='EOP_calibrate',kwargs_init={'check_variables':True},overwrite=True,kwargs_mi=kwargs_mi)
db_EOP_C = gm.model_instances['EOP_calibrate'].out_db

## A: Sneaky solve with non-default solve statement

*Reset to EOP state:*

In [13]:
gm.reset_settings()
gm.setstate('EOP',init=False)
gm.write_and_run(name='EOP',kwargs_init={'check_variables':True},overwrite=True,kwargs_mi=kwargs_mi)

*set up target database:*

In [14]:
db_star = DataBase.GPM_database()
db_star['sigmaG'] = gm.get('sigmaG')/10
db_star['sigma'] = pd.Series(5, index = gm.get('kno_ID_CU'),name='sigma')

In [15]:
gm.model_instances['EOP'].solve_sneakily(db_star=db_star)

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [16]:
db = gm.model_instances['EOP'].out_db

*Solve sneakily:*

In [17]:
# gm.model_instances['EOP_calibrate'].solve_sneakily(db_star=db_star, kwargs_shock = {'solvetext': gm.model.settings.get_conf('solve')})

*Set up baseline model and solve:*

In [18]:
# gm.write_and_run(name='EOP',kwargs_mi=kwargs_mi)
# db_EOP = gm.model_instances['EOP'].out_db